In [1]:
import os, sys

root = os.path.dirname(os.getcwd())
sys.path.append(root)
print(f"{root = }")
    
os.environ["CUDA_VISIBLE_DEVICES"] = '5'

root = '/home/coder/projects/test/keyword'


In [2]:
import mlflow
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM,
    EarlyStoppingCallback,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import (
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model,
    PeftModel
)

from src.utils import Log, helper_function

In [3]:
# set seed and cudnn
helper_function.set_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [4]:
# get logger
log = Log()
logger = log.set_logger(file_path = f"{root}/logs/log.log", level = 1, freq = "D", interval = 10, backup = 3, name = "log")

paths = {
    'data': f'{root}/data',
    'reports': f'{root}/reports',
    'models': f'{root}/models',
    'src': f'{root}/src',
}

params = {
    'init': {
            'paths': paths,
        },
}

In [5]:
class Train():
    def __init__(self, paths, logger):
        self.paths = paths
        self.logger = logger
    def setting_mlflow(self):
        """
        setting mlflow configuration to record training process
        """
        mlflow.set_tracking_uri(self.exp_config['mlflow']['tracking_uri'])
        mlflow.set_experiment(self.exp_config['mlflow']['experiment_name'])
        mlflow.start_run(run_name=self.exp_config['mlflow']['run_name'])
        mlflow.set_tags(self.exp_config['mlflow']['tags'])
        os.environ['LOGNAME'] = self.exp_config['mlflow']['log_name']

    def finish_mlflow(self):
        """
        finish mlflow run
        """
        mlflow.end_run()

    def log_to_mlflow(self, model, tokenizer, log_model_flag = False):
        """
        log model to mlflow server
        """
        if log_model_flag:
            # log model to mlflow server
            components = {
                "model": model,
                "tokenizer": tokenizer
                }
            mlflow.transformers.log_model(
                transformers_model=components,
                artifact_path="tagging_model",
            )
        else:
            # log model path
            model_path = f"{self.paths['models']}/{self.exp_config['model']['name']}/{self.exp_config['model']['description']}"
            mlflow.log_param("model_path", model_path)

        # log config file
        mlflow.log_artifact(f"{root}/config/experiment/config.yml")

# class object
self = Train(**params["init"], logger=logger)

In [6]:
self.exp_config = helper_function.load_config(f"{paths['src']}/config/experiment/config.yml")
self.exp_config

{'mlflow': {'tracking_uri': 'http://localhost:5000',
  'experiment_name': 'story-structure',
  'run_name': 'test',
  'log_name': 'julie_kuo',
  'device_id': '5',
  'tags': {'base_model': 'MediaTek-Research/Breeze-7B-Instruct-v0_1',
   'type': 'CAUSAL_LM',
   'loss': 'cross_entropy',
   'dataset': 'percy_jackson'}},
 'data': {'file_path': '/raw/percy_jackson.txt'},
 'model': {'name': 'MediaTek-Research/Breeze-7B-Instruct-v0_1',
  'description': 'breeze',
  'checkpoint': 'checkpoint-300',
  'chunk_size': 256,
  'early_stopping_patience': 20,
  'seed': 42},
 'training_args': {'eval_steps': 20,
  'save_steps': 100,
  'logging_steps': 20,
  'max_steps': 5000,
  'save_total_limit': 20,
  'per_device_train_batch_size': 2,
  'per_device_eval_batch_size': 2,
  'gradient_accumulation_steps': 8,
  'gradient_checkpointing': True,
  'gradient_checkpointing_kwargs': {'use_reentrant': False},
  'warmup_steps': 100,
  'learning_rate': 0.0001,
  'optim': 'paged_adamw_8bit',
  'logging_strategy': 'steps

In [7]:
self.setting_mlflow()

### Fine-tuning model with peft

In [8]:
def load_data(file_path):
    """載入並預處理文本數據"""
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    
    # 將文本分割成較小的段落，使用較短的長度
    self.chunk_size = self.exp_config["model"]["chunk_size"]
    chunks = [text[i:i+self.chunk_size] for i in range(0, len(text), self.chunk_size)]
    
    # 創建dataset
    dataset = Dataset.from_dict({
        'text': chunks
    })
    
    return dataset

In [9]:
def prepare_model_and_tokenizer(model_name):
    """準備小型基礎模型和分詞器，使用新的量化配置"""
    # 設定 4bit 量化配置
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,  # 啟用 4bit 載入，以減少模型大小和記憶體使用量
        bnb_4bit_quant_type="nf4",  # 指定量化類型
        bnb_4bit_compute_dtype=torch.float16,  # 設定計算數據類型，以加快運算速度
        bnb_4bit_use_double_quant=True  # 啟用雙重量化，提高準確性但略增加記憶體用量
    )
    
    # 載入模型和分詞器
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,  # 使用新的量化配置
        torch_dtype=torch.float16,  # 設定模型的預設數據類型，以優化記憶體使用量
        device_map="auto"  # 自動分配模型到可用的設備上（CPU或GPU）
    )
    
    # 配置 LoRA
    lora_config = LoraConfig(
        r=8,  # LoRA的秩，控制LoRA矩陣的大小，影響參數數量和計算複雜度
        lora_alpha=16,  # LoRA放大因子，調整LoRA參數的學習率 (可以先設成 2倍 r)
        target_modules=[
            "q_proj",    # 查詢投影
            "k_proj",    # 鍵值投影
            "v_proj",    # 數值投影
            "o_proj",    # 輸出投影
            "gate_proj", # 門控投影
            "up_proj",   # 上投影
            "down_proj"  # 下投影
        ],  # 指定 LoRA 作用的模型部分
        lora_dropout=0.05,  # 設定 LoRA 層的 dropout 率，用於防止過擬合
        bias="none",  # 關閉偏差項
        task_type="CAUSAL_LM"  # 指定任務類型為因果語言模型
    )
    
    model = prepare_model_for_kbit_training(model)  # 準備模型以支持 kbit 訓練
    model = get_peft_model(model, lora_config)  # 使用 LoRA 配置加速模型訓練    

    # move model to available device
    available_device = torch.device(
        'cuda' if torch.cuda.is_available() else 'cpu')
    model.to(available_device)
    
    return model, tokenizer


In [10]:
def tokenize_function(examples, tokenizer):
    """將文本轉換為token"""
    return tokenizer(
        examples["text"],
        truncation=True, # 截斷文本以符合模型的最大長度
        max_length=self.chunk_size,  # 設定最大長度
        padding="max_length" # 填充文本以符合模型的最大長度
    )

In [11]:
# 載入數據
dataset = load_data(f'{self.paths["data"]}/{self.exp_config["data"]["file_path"]}')

# 分割訓練集和驗證集
train_test_split = dataset.train_test_split(test_size=0.1, seed=self.exp_config["model"]["seed"], shuffle=True)
train_dataset_raw = train_test_split['train']
valid_dataset_raw = train_test_split['test']

In [ ]:
# 準備模型和分詞器
model, tokenizer = prepare_model_and_tokenizer(self.exp_config["model"]["name"])

In [ ]:
# 處理數據集
train_dataset = train_dataset_raw.map(
    lambda x: tokenize_function(x, tokenizer), # 將文本轉換為token
    batched=True, # 一次處理多個樣本
    remove_columns=dataset.column_names # 移除原始文本列
)

# 處理數據集
valid_dataset = valid_dataset_raw.map(
    lambda x: tokenize_function(x, tokenizer), # 將文本轉換為token
    batched=True, # 一次處理多個樣本
    remove_columns=dataset.column_names # 移除原始文本列
)

In [14]:
# create model checkpoint folder
model_path = f"{self.paths['models']}/{self.exp_config['model']['name']}/{self.exp_config['model']['description']}"
helper_function.remove_directory(model_path) # remove old model checkpoint
os.makedirs(model_path, exist_ok=True) # create new model checkpoint

# create logging folder
logging_dir = f'{self.paths["reports"]}/{self.exp_config["model"]["name"]}/{self.exp_config["model"]["description"]}/logs'
helper_function.remove_directory(model_path) # remove old model checkpoint
os.makedirs(logging_dir, exist_ok=True) # create new model checkpoint

In [ ]:
# 訓練參數配置
training_args = TrainingArguments(
    output_dir = model_path, 
    logging_dir = logging_dir,
    device_map={"": 0}, # 指定模型放置在GPU (強制所有參數都在 GPU 0 上)
    **self.exp_config['training_args']
    )

In [16]:
# 設定數據整理器
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False # 是否隨機遮蔽token，關閉以進行因果語言模型訓練，即預測下一個token
)

In [17]:
# 創建訓練器
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    callbacks=[EarlyStoppingCallback(self.exp_config['model']['early_stopping_patience'])]
    # compute_metrics=compute_metrics, # 添加計算指標的函數
    # loss_fn=custom_loss,
)

In [ ]:
gpu_dict = helper_function.check_device(model, trainer.args)
self.logger.info(f"-- Model is using {'GPU: ' + str(gpu_dict['device_idx']) if gpu_dict['device_idx'] != -1 else 'CPU'}")
self.logger.info(f"-- training_args is using {gpu_dict['model_gpu_idx']}")

In [ ]:
# 開始訓練
trainer.train()

In [ ]:
self.finish_mlflow()

In [ ]:
# log model, tokenizer and config file to mlflow
self.log_to_mlflow(model, tokenizer, log_model_flag = False)
self.finish_mlflow()

### Test model response

In [7]:
def load_original_model(model_name):
    """載入原始模型"""    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto"
    )
    
    return model, tokenizer

In [8]:
def load_finetuned_model(model_name, model_description, checkpoint):
    """載入微調後的模型"""
    adapter_path = f"{root}/models/{model_name}/{model_description}/{checkpoint}"
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    # 載入基礎模型
    base_model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto"
    )
    
    # 載入 LoRA 權重
    model = PeftModel.from_pretrained(base_model, adapter_path)
    model = model.merge_and_unload() # 合併 LoRA 權重並釋放記憶體
    
    return model, tokenizer

In [9]:
def generate_response(model, tokenizer, prompt, max_length=512):
    """生成回應"""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.inference_mode():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_return_sequences=1,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [10]:
question = "你看過波西傑克森這本關於希臘神話的小說嗎?看過的話說明一下故事主軸。"

In [ ]:
print("=== 原始模型回應 ===")
model, tokenizer = load_original_model(self.exp_config["model"]["name"])
original_response = generate_response(model, tokenizer, question)
print(original_response)

In [11]:
print("=== 微調後模型回應 ===")
model, tokenizer = load_finetuned_model(
    self.exp_config["model"]["name"], 
    self.exp_config["model"]["description"], 
    self.exp_config["model"]["checkpoint"]
    )
finetuned_response = generate_response(model, tokenizer, question)
print(finetuned_response)

=== 微調後模型回應 ===


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

你看過波西傑克森這本關於希臘神話的小說嗎?看過的話說明一下故事主軸。」
「波西·傑克遜。」
「對，他寫的是關於『十二神諭』。這故事發生在第二次世界大戰的時候，就是一九四○年。當時的奧林匹斯山是一個神秘的地方，因為人類已經忘記了希臘神話。奧林匹斯山的存在在人類的意識裡已經變得模糊了。而波西·傑克遜在美國西部的一個小鎮裡遇到了一些神秘的力量，然後，他回到紐約。」
「然後發生了什麼？」
「他發現奧林匹斯山正被某個邪惡的力量襲擊，這個力量正想摧毀奧林匹斯山，消滅眾神。」
「這個邪惡的力量是誰？」
「他就是克里奧斯·羅德斯，克洛諾斯的兒子。」
「克洛諾斯的兒子？克洛諾斯是什麼人？」
「克洛諾斯是諸神的父親，就是我們所說的海神。克洛諾斯是一個巨大的巨人，他曾試圖擊敗眾神。在一戰中，他被諸神所滅，但克洛諾斯的靈魂並沒有完全消滅。」
「你剛才說的這個克里奧斯·羅德斯是克洛諾斯的兒子，那他的父親是……」
「就是克洛諾斯，海神的兒子。」
「可是克洛諾斯已經死了啊！」
「沒錯，但克洛諾斯並沒有死，他只是被封印了。他仍然活在奧林匹斯山。克里奧斯·羅德斯是克洛諾斯的兒子，他繼承了父親的力量。克洛諾斯曾經把奧林匹斯山的存在隱藏了，因此，奧林匹斯山對於人類來說成了一個神秘的地方。」
「所以奧林匹斯山現在還存在嗎？」
「是的，它依然存在。波西·傑克遜拯救了奧林匹斯山，阻止了克里奧斯·羅德斯的計劃。克里奧斯·羅德斯的計劃是摧毀奧林匹斯山，並且利用他的父親克洛諾斯的力量來征服世界。」
「這就是波塞冬的故事嗎？」
「不，波塞冬的故事是另外一個故事。波塞冬是海洋之神。他和克洛諾斯


In [12]:
question = '''
選項: 1. 離開 2. 說服 3. 戰鬥
選擇: 2. 說服

產生後續情節
'''

finetuned_response = generate_response(model, tokenizer, question)
print(finetuned_response)


選項: 1. 離開 2. 說服 3. 戰鬥
選擇: 2. 說服

產生後續情節

我轉身面對斯隆。「我知道你為什麼在這兒。」
「你知道嗎？」
「你想要的不是我的生命。你已經得到了。你想要的是……」
「我想要你的父親的生命。」
「你為什麼？」
斯隆的表情變得很嚴肅，彷彿在努力讓自己保持平靜。「因為他……他背叛了我。他背叛了所有的神祇。他背叛了克洛諾斯。他背叛了我們所有人。」
「所以你打算殺了他。」
「是的。」
「你知道他並沒有背叛你嗎？」
「我必須這麼做。」
「你真的相信克洛諾斯嗎？」
斯隆的眼睛發出光芒。「是的，波西，我相信他。我相信他會帶領我們打敗奧林匹斯神。」
「可你自己也是奧林匹斯神，斯隆。」
「你知道那是怎麼回事嗎？」
「你已經死過一次。你已經成為克洛諾斯的奴隸了。」
斯隆的目光變得銳利而冷漠。「我不知道你怎麼知道的。」
「波塞冬告訴我的。」
斯隆的臉色變得陰沉下來，彷彿我剛戳到了一個敏感的問題。「波塞冬，他只是個普通的海洋神祇。他什麼都沒有。」
「可是他告訴我……」
「他只是在胡說八道。我必須去殺死波塞冬，然後克洛諾斯就會征服所有的神祇。」
「你知道他不是真的，斯隆。」
「波西，我必須去。你只能阻止我，或者……」
「或者什麼？」
「或者，你可以幫助我。」
「幫助你幹什麼？」
「去找到我的父親。去告訴他，告訴他克洛諾斯回來了。」
「我告訴你，斯隆。克洛諾斯並沒有回來。」
斯隆的目光從我的身上移開，朝了遠處。「波西，你應該走開了。」
「我不能走，斯隆。你需要我的幫助。」
「波西，你已經幫助我了。」
